In [ ]:
'''
Resource to rebuild https://stackabuse.com/accessing-the-twitter-api-with-python/
'''


In [2]:
import pandas as pd
import numpy as np
import json
import csv
import ast
import gmplot

from twython import Twython
from twython import TwythonStreamer
from collections import Counter
from geopy.geocoders import Nominatim


In [28]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': 'premier league',
        #'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [29]:
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])



In [31]:
# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
print(df)

            user                            date  \
0        bephyne  Sun Mar 08 19:17:09 +0000 2020   
63      Abiasis9  Sun Mar 08 19:16:44 +0000 2020   
73       eDeejae  Sun Mar 08 19:16:39 +0000 2020   
72      supabara  Sun Mar 08 19:16:39 +0000 2020   
71  BekwaiNative  Sun Mar 08 19:16:40 +0000 2020   
..           ...                             ...   
30     Lakwafo_7  Sun Mar 08 19:16:56 +0000 2020   
29   Scott_Ball_  Sun Mar 08 19:16:56 +0000 2020   
28  Kwame_Domani  Sun Mar 08 19:16:57 +0000 2020   
27    asiwajudin  Sun Mar 08 19:16:57 +0000 2020   
99     brunosznn  Sun Mar 08 19:16:31 +0000 2020   

                                                 text  favorite_count  
0   RT @MidKnightGaz: These two young men grew up ...               0  
63  @TheRedmenTV Jurgen, how many pionts do we nee...               0  
73  RT @billygilmourrr: Proud moment to make my fu...               0  
72  RT @SkySportsPL: Alexandre Lacazette had VAR t...               0  
71  RT @GHPLUpd

To explore data back in history, you'll need to purchase the Premium or Enterprise plan of the Search API.

Create an streaming process

In [33]:

tweets = pd.read_csv("saved_tweets.csv")
tweets['text'].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())
tweets.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 115: invalid start byte

In [3]:
tweets = pd.read_csv("saved_tweets.csv")

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets.hashtags]
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)
counter_hashtags.most_common(20)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x85 in position 115: invalid start byte

In [ ]:
geolocator = Nominatim()

# Go through all tweets and add locations to 'coordinates' dictionary
coordinates = {'latitude': [], 'longitude': []}
for count, user_loc in enumerate(tweets.location):
    try:
        location = geolocator.geocode(user_loc)
        
        # If coordinates are found for location
        if location:
            coordinates['latitude'].append(location.latitude)
            coordinates['longitude'].append(location.longitude)
            
    # If too many connection requests
    except:
        pass
    
# Instantiate and center a GoogleMapPlotter object to show our map
gmap = gmplot.GoogleMapPlotter(30, 0, 3)

# Insert points on the map passing a list of latitudes and longitudes
gmap.heatmap(coordinates['latitude'], coordinates['longitude'], radius=20)

# Save the map to html file
gmap.draw("python_heatmap.html")